In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directories for training and validation data
train_dir = r'C:\Users\anshu\Emoji-Generator\data\train'
val_dir = r'C:\Users\anshu\Emoji-Generator\data\test'

# Image data generators for preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

# Create validation generator
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

# Define the model
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

# Disable OpenCL for OpenCV
cv2.ocl.setUseOpenCL(False)

# Emotion dictionary
emotion_dict = {
    0: "Angry", 
    1: "Disgusted", 
    2: "Fearful", 
    3: "Happy", 
    4: "Neutral", 
    5: "Sad", 
    6: "Surprised"
}

# Compile the model
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Fit the model
emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)



Found 27544 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 126s 272ms/step - accuracy: 0.2530 - loss: 1.8124 - val_accuracy: 0.2422 - val_loss: 1.8385
Epoch 2/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.2500 - loss: 1.7722 - val_accuracy: 0.0000e+00 - val_loss: 2.0985
Epoch 3/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 123s 285ms/step - accuracy: 0.2995 - loss: 1.7360 - val_accuracy: 0.3378 - val_loss: 1.7140
Epoch 4/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 219us/step - accuracy: 0.2500 - loss: 1.6866 - val_accuracy: 0.6000 - val_loss: 1.2252
Epoch 5/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 104s 240ms/step - accuracy: 0.3422 - loss: 1.6411 - val_accuracy: 0.3171 - val_loss: 1.6336
Epoch 6/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 182us/step - accuracy: 0.3594 - loss: 1.4698 - val_accuracy: 0.4000 - val_loss: 1.4608
Epoch 7/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 130s 301ms/step - accuracy: 0.3778 - loss: 1.5603 - val_accuracy: 0.3297 - val

In [1]:
# Save model weights
# Save model weights
emotion_model.save_weights('emotion_model2.weights.h5')

NameError: name 'emotion_model' is not defined

In [2]:

# Start video capture for real-time emotion detection
cap = cv2.VideoCapture(0)
while True:
    # Read the frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Load Haar cascade for face detection
    bounding_box = cv2.CascadeClassifier(r'C:\Users\anshu\Emoji-Generator\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined